In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import importlib
import sys

from tests.metaworld.envs.mujoco.sawyer_xyz import utils
sys.path.append('/home/hendrik/Documents/master_project/Code/LanguagePolicies/')
from model_src.modelTorch import PolicyTranslationModelTorch
from utils.Transformer import TailorTransformer
from utils.networkMeta import NetworkMeta
from utils.networkTorch import NetworkTorch
import hashids
import time
import numpy as np
import torch
import torch.nn as nn
from prettytable import PrettyTable
import sys
import pickle
from utilsMW.model_setup import model_setup
from utilsMW.dataLoaderMW import TorchDatasetMWToy
from torch.utils.data import DataLoader
from gym import logger
from utilsMW.makeTrainingData import ToySimulation
from searchTest.toyEnvironment import make_func, check_outpt, make_tol


# Learning rate for the adam optimizer
LEARNING_RATE   = 5e-6
# Weight for the attention loss
WEIGHT_ATTN     = 1.0
# Weight for the motion primitive weight loss
WEIGHT_W        = 1.0
# Weight for the trajectroy generation loss
WEIGHT_TRJ      = 1#5.0

WEIGHT_GEN_TRJ  = 50

# Weight for the time progression loss
WEIGHT_DT       = 14.0
# Weight for the phase prediction loss
WEIGHT_PHS      = 1 #1.0

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

2022-05-03 10:36:54.269507: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/hendrik/Documents/master_project/Code/LanguagePolicies/ros2/src/install/policy_translation/lib:/opt/ros/foxy/opt/yaml_cpp_vendor/lib:/opt/ros/foxy/opt/rviz_ogre_vendor/lib:/opt/ros/foxy/lib/x86_64-linux-gnu:/opt/ros/foxy/lib::/home/hendrik/Documents/master_project/CoppeliaSim_Player_V4_1_0_Ubuntu20_04:/home/hendrik/.mujoco/mujoco200/bin
2022-05-03 10:36:54.269524: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/hendrik/anaconda3/envs/mujoco/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [ ]:




def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

def setupModel(device , epochs ,  batch_size, path_dict , logname , model_path, tboard, model_setup, train_size = 1):
    train_path = path_dict['META_WORLD'] + 'train/'
    val_path = path_dict['META_WORLD'] + 'val/'
    test_path = path_dict['META_WORLD'] + 'test/'
    train_data = TorchDatasetMWToy(path=train_path, device=device)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_data = TorchDatasetMWToy(path=val_path, device=device)
    eval_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
    test_data = TorchDatasetMWToy(path=test_path, device=device)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)
    env_tag = 'pickplace'
    for inpt, outpt in train_loader:
        dim_in = inpt.size(1)
        dim_out = outpt.size(-1)
        seq_len = outpt.size(1)
        print(inpt.shape)
        break
    model_setup['plan_nn']['plan']['d_output'] = dim_out
    model_setup['plan_nn']['plan']['seq_len'] = seq_len
    model   = PolicyTranslationModelTorch(od_path="", model_setup=model_setup, device=device).to(device)
    for inpt, outpt in train_loader:
        result = model(inpt)
        break
    tol_neg, tol_pos = make_tol(std_dev=5e-3, dim=dim_out, add=0.8, device='cuda')
    successSimulation = ToySimulation(neg_tol=tol_neg, pos_tol=tol_pos, check_outpt_fct=check_outpt, dataloader=test_loader)

    model_setup['tailor_transformer']['seq_len'] = seq_len
    tailor_model = TailorTransformer(model_setup=model_setup['tailor_transformer'])
    
    
    network = NetworkMeta(model, tailor_models=[tailor_model], env_tag=env_tag, successSimulation=successSimulation, data_path=path_dict['DATA_PATH'],logname=logname, lr=LEARNING_RATE, lw_atn=WEIGHT_ATTN, lw_w=WEIGHT_W, lw_trj=WEIGHT_TRJ, lw_gen_trj = WEIGHT_GEN_TRJ, lw_dt=WEIGHT_DT, lw_phs=WEIGHT_PHS, lw_fod=0, gamma_sl = 1, device=device, tboard=tboard)
    network.setDatasets(train_loader=train_loader, val_loader=eval_loader)

    network.setup_model(model_params=model_setup)
    if model_path is not None:
        model.load_state_dict(torch.load(model_path, map_location='cuda:0'))
    count_parameters(network)
    #print('in tailor transfo:')
    #count_parameters(tailor_model)

    network.train(epochs=epochs, model_params=model_setup)
    return network

import os

logger.set_level(40)
args = [
    '-path', '/home/hendrik/Documents/master_project/LokalData/', '-batch_size', '32', '-train_size', '1', '-epochs', '1000', '-tboard', 'False'
]
if '-path' not in args:
    print('no path given, not executing code')
else:    
    data_path = args[args.index('-path') + 1]
    path_dict = {
    'TRAIN_DATA_TORCH' : os.path.join(data_path, 'TorchDataset/train_data_torch.txt'),
    'VAL_DATA_TORCH' : os.path.join(data_path, 'TorchDataset/val_data_torch.txt'),
    'TRAIN_DATA' : os.path.join(data_path, 'GDrive/train.tfrecord'),
    'VAL_DATA' : os.path.join(data_path, 'GDrive/validate.tfrecord'),
    'GLOVE_PATH' : os.path.join(data_path, 'GDrive/glove.6B.50d.txt'),
    'META_WORLD' : os.path.join(data_path, '/home/hendrik/Documents/master_project/LokalData/metaworld/test/toy_data/'),
    'DATA_PATH' : data_path
    }

    device = 'cuda'
    if '-device' in args:
        device = args[args.index('-device') + 1]
    from utilsMW.toy_model_setup import model_setup
    model_path = None
    if '-model' in args:
        model_path = args[args.index('-model') + 1] + 'policy_translation_h'
        if '-model_setup' in args:
            setup_path = args[args.index('-model') + 1] + 'model_setup.pkl'
            with open(setup_path, 'rb') as f:
                model_setup = pickle.load(f)
            model_setup['train']      = True
            print('load model')
    model_setup = model_setup
    
    epochs = 200
    if '-epochs' in args:
        epochs = int(args[args.index('-epochs') + 1])

    batch_size = 16
    if '-batch_size' in args:
        batch_size = int(args[args.index('-batch_size') + 1])

    tboard = True
    if '-tboard' in args:
        tboard = (args[args.index('-tboard') + 1]) == 'True'
        print(f'tboard: {tboard}')

    train_size = 1
    if '-train_size' in args:
        train_size = float(args[args.index('-train_size') + 1])

    hid             = hashids.Hashids()
    logname         = hid.encode(int(time.time() * 1000000))
    print(f'logname: {logname}')
    network = setupModel(device=device, epochs = epochs, batch_size = batch_size, path_dict = path_dict, logname=logname, model_path=model_path, tboard=tboard, model_setup=model_setup, train_size=train_size)
    print(f'end saving: {path_dict["MODEL_PATH"]}')
    torch.save(network.state_dict(), path_dict['MODEL_PATH'])



In [7]:
from __future__ import absolute_import, division, print_function, unicode_literals
import importlib
import sys

from tests.metaworld.envs.mujoco.sawyer_xyz import utils
sys.path.append('/home/hendrik/Documents/master_project/Code/LanguagePolicies/')
from model_src.modelTorch import PolicyTranslationModelTorch
from utils.Transformer import TailorTransformer
from utils.networkMeta import NetworkMeta
from utils.networkTorch import NetworkTorch
from utilsMW.dataLoaderMW import TorchDatasetTailor
import hashids
import time
import numpy as np
import torch
import torch.nn as nn
from prettytable import PrettyTable
import sys
import pickle
from utilsMW.toy_model_setup import model_setup
from utilsMW.dataLoaderMW import TorchDatasetMWToy
from torch.utils.data import DataLoader
from gym import logger
from utilsMW.makeTrainingData import ToySimulation
from searchTest.toyEnvironment import make_func, check_outpt, make_tol

model_setup['tailor_transformer']
tailor_model = TailorTransformer(model_setup=model_setup['tailor_transformer'])


trajectories = torch.rand([10,1,1]).to('cuda')
obsv = torch.rand([10,1,1]).to('cuda')
success = torch.ones((10)).to('cuda')
success[:2] = 0
#success[1] = 0

tailor_data = TorchDatasetTailor(trajectories= trajectories, obsv=obsv, success=success)
tailor_loader = DataLoader(tailor_data, batch_size=2, shuffle=True)

for outpt in tailor_loader:
    succ, failed = outpt
    s_trj, s_obs, success = succ
    f_trj, f_obs, fail = failed
    trajectories = torch.cat((s_trj, f_trj), dim=0)
    inpt = torch.cat((s_obs, f_obs), dim=0)
    label = torch.cat((success, fail), dim=0).type(torch.long)
    label_one_hot = torch.nn.functional.one_hot(label, num_classes = 2)
    inpt_super = torch.concat((trajectories, inpt), dim = -1)
    output = tailor_model.forward(inpt_super)
    break
optimizer = torch.optim.Adam(tailor_model.parameters(), lr=1e-4)
for i in range(10000):
    avrg_loss = 0
    for outpt in tailor_loader:
        succ, failed = outpt
        s_trj, s_obs, success = succ
        f_trj, f_obs, fail = failed
        trajectories = torch.cat((s_trj, f_trj), dim=0)
        inpt = torch.cat((s_obs, f_obs), dim=0)
        label = torch.cat((success, fail), dim=0).type(torch.long)
        label_one_hot = torch.nn.functional.one_hot(label, num_classes = 2)
        inpt_super = torch.concat((trajectories, inpt), dim = -1)
        output = tailor_model.forward(inpt_super)
        loss = ((output.reshape(-1) - label_one_hot.reshape(-1))**2).mean()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        avrg_loss+=loss
    if i%100==0:
        print(avrg_loss)
    




tensor(0.9820, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.9637, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.8567, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.7925, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.6124, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.6801, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.8451, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.4393, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.4365, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.7328, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.6033, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.4842, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.3821, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.4112, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5367, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.3793, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.3954, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.4218, device='cuda:0', grad_fn=<AddBack

KeyboardInterrupt: 

In [31]:
from utilsMW.dataLoaderMW import TorchDatasetTailor
from torch.utils.data import DataLoader

trajectories = torch.rand([10,1,1])
obsv = torch.rand([10,1,1])
success = torch.ones((10))
success[0] = 0
#success[1] = 0

tailor_data = TorchDatasetTailor(trajectories= trajectories, obsv=obsv, success=success)
tailor_loader = DataLoader(tailor_data, batch_size=2, shuffle=True)


In [32]:
trajectories

tensor([[[0.9463]],

        [[0.7719]],

        [[0.7471]],

        [[0.4551]],

        [[0.5530]],

        [[0.1816]],

        [[0.1539]],

        [[0.3654]],

        [[0.7727]],

        [[0.9233]]])

In [33]:
obsv

tensor([[[0.8743]],

        [[0.0692]],

        [[0.0792]],

        [[0.3569]],

        [[0.3110]],

        [[0.7282]],

        [[0.0724]],

        [[0.7933]],

        [[0.6466]],

        [[0.5477]]])

In [42]:
for outpt in tailor_loader:
    succ, failed = outpt
    s_trj, s_obs, success = succ
    f_trj, f_obs, fail = failed
    trajectories = torch.cat((s_trj, f_trj), dim=0)
    inpt = torch.cat((s_obs, f_obs), dim=0)
    label = torch.cat((success, fail), dim=0).type(torch.long)
    label_one_hot = torch.nn.functional.one_hot(label, num_classes = 2)
    


torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([2, 2])
